In [12]:
from src import *

from functools import partial
from multiprocessing import Pool

from brian2 import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings("ignore")

In [14]:
prefs.codegen.target = "numpy"
start_scope()
np.random.seed(100)
data_path = './data/MNIST/raw/' # '../../../Data/MNIST_data/'


In [64]:
# -----simulation parameter setting-------
coding_n = 3           # cos编码基函数数量
MNIST_shape = (1, 28*28)
coding_duration = 30   # 编码序列的长度
duration = coding_duration * MNIST_shape[0]
F_train = 0.05
F_validation = 0.00833333
F_test = 0.05
Dt = defaultclock.dt = 1 * ms

# -------class initialization----------------------
function = MathFunctions()
base = BaseFunctions()
readout = Readout()
MNIST = MNIST_classification(MNIST_shape, duration)

# -------data initialization----------------------
MNIST.load_Data_MNIST_all(data_path)
df_train_validation = MNIST.select_data(F_train + F_validation, MNIST.train)
df_train, df_validation = train_test_split(df_train_validation, 
                                           test_size=F_validation / (F_validation + F_train),
                                           random_state=42)
df_test = MNIST.select_data(F_test, MNIST.test)

df_en_train = MNIST.encoding_latency_MNIST(MNIST._encoding_cos_rank_ignore_0, df_train, coding_n)
df_en_validation = MNIST.encoding_latency_MNIST(MNIST._encoding_cos_rank_ignore_0, df_validation, coding_n)
df_en_test = MNIST.encoding_latency_MNIST(MNIST._encoding_cos_rank_ignore_0, df_test, coding_n)

data_train_s, label_train = MNIST.get_series_data_list(df_en_train, is_group=True)
data_validation_s, label_validation = MNIST.get_series_data_list(df_en_validation, is_group=True)
data_test_s, label_test = MNIST.get_series_data_list(df_en_test, is_group=True)

# -------get numpy random state------------
np_state = np.random.get_state()


In [98]:
a = [(x) for x in zip(data_train_s, label_train)]
len(a)

3000

In [71]:
2352/28/28

3.0

In [87]:
# ---- define network run function----

"""
run_net(inputs, parameter)
    Parameters = [R, p_inE/I, f_in, f_EE, f_EI, f_IE, f_II, tau_ex, tau_inh]
    
    f_in: 线性输入层——池化层的突触连接强度参数
    f_EE: 兴奋性——兴奋性,突触强度参数
    f_EI: 兴奋性——抑制性,突触强度参数
    f_IE:
    f_II:
    tau_ex: 兴奋性神经元的膜时间常数
    tau_inh: 抑制性神经元的膜时间常数
"""
parameter = {
    'R':    0.5,
    'f_in': 0.5,
    'f_EE': 0.5,
    'f_EI': 0.5,
    'f_IE': 0.5,
    'f_II': 0.5,
    'tau_ex': 0.5,
    'tau_inh': 0.5,
    'p_in': 0.5,
}

In [88]:
# ---- set numpy random state for each run----
np.random.set_state(np_state)

# -----parameter setting-------
n_ex = 1600             # 兴奋性神经元数量
n_inh = int(n_ex / 4)   # 抑制性神经元数量
n_input = MNIST_shape[1] * coding_n # 784*3=2352
n_read = n_ex + n_inh

R = parameter['R']
f_in = parameter['f_in']
f_EE = parameter['f_EE']
f_EI = parameter['f_EI']
f_IE = parameter['f_IE']
f_II = parameter['f_II']

A_EE = 60 * f_EE
A_EI = 60 * f_EI
A_IE = 60 * f_IE
A_II = 60 * f_II
A_inE = 60 * f_in
A_inI = 60 * f_in

tau_ex = parameter['tau_ex'] * coding_duration
tau_inh = parameter['tau_inh'] * coding_duration
tau_read = 30

p_inE = parameter['p_in'] * 0.1 # 线性输入层神经元与兴奋性神经元的连接概率
p_inI = parameter['p_in'] * 0.1 # 线性输入层神经元与抑制性神经元的连接概率

In [89]:
# ------definition of equation-------------
neuron_in = '''
I = stimulus(t,i) : 1
'''

neuron = '''
tau : 1
dv/dt = (I-v) / (tau*ms) : 1 (unless refractory)
dg/dt = (-g)/(3*ms) : 1
dh/dt = (-h)/(6*ms) : 1
I = (g+h)+13.5: 1
x : 1
y : 1
z : 1
'''

neuron_read = '''
tau : 1
dv/dt = (I-v) / (tau*ms) : 1
dg/dt = (-g)/(3*ms) : 1 
dh/dt = (-h)/(6*ms) : 1
I = (g+h): 1
'''

synapse = '''w : 1'''

on_pre_ex = '''g+=w''' # 兴奋性神经元，作为突触前神经元发放，将执行的代码

on_pre_inh = '''h-=w''' # 抑制性神经元，作为突触前神经元发放，将执行的代码

In [90]:
# -----Neurons setting-------
Input = NeuronGroup(n_input, 
                    neuron_in, 
                    threshold='I > 0', 
                    method='euler', 
                    refractory=0 * ms,
                    name='neurongroup_input')

# excitatory兴奋性神经元
G_ex = NeuronGroup(n_ex, 
                    neuron, 
                    threshold='v > 15', 
                    reset='v = 13.5', 
                    method='euler', 
                    refractory=3 * ms,
                    name='neurongroup_ex')

# inhibitory异质性神经元
G_inh = NeuronGroup(n_inh, 
                    neuron, 
                    threshold='v > 15', 
                    reset='v = 13.5', 
                    method='euler', 
                    refractory=2 * ms,
                    name='neurongroup_in')

# 读出层
G_readout = NeuronGroup(n_read, neuron_read, method='euler', name='neurongroup_read')

In [91]:
# -----Synapses setting-------
# 输入层——兴奋性
S_inE = Synapses(Input, G_ex, synapse, on_pre=on_pre_ex, method='euler', name='synapses_inE')
# 输入层——抑制性
S_inI = Synapses(Input, G_inh, synapse, on_pre=on_pre_inh, method='euler', name='synapses_inI')
# 兴奋性——兴奋性
S_EE = Synapses(G_ex, G_ex, synapse, on_pre=on_pre_ex, method='euler', name='synapses_EE')
# 兴奋性——抑制性
S_EI = Synapses(G_ex, G_inh, synapse, on_pre=on_pre_ex, method='euler', name='synapses_EI')
# 抑制性——兴奋性
S_IE = Synapses(G_inh, G_ex, synapse, on_pre=on_pre_inh, method='euler', name='synapses_IE')
# 抑制性——抑制性
S_II = Synapses(G_inh, G_inh, synapse, on_pre=on_pre_inh, method='euler', name='synapses_I')
# 兴奋性——读出层
S_E_readout = Synapses(G_ex, G_readout, 'w = 1 : 1', on_pre=on_pre_ex, method='euler', name='synapses_Er')
# 抑制性——读出层
S_I_readout = Synapses(G_inh, G_readout, 'w = 1 : 1', on_pre=on_pre_inh, method='euler', name='synapses_Ir')

# -------initialization of neuron parameters----------
G_ex.v = '13.5+1.5*rand()'
G_inh.v = '13.5+1.5*rand()'
G_readout.v = '0'

G_ex.g = '0'
G_inh.g = '0'
G_readout.g = '0'

G_ex.h = '0'
G_inh.h = '0'
G_readout.h = '0'

G_ex.tau = tau_ex # 时间常数
G_inh.tau = tau_inh
G_readout.tau = tau_read
# 给神经元分配XYZ坐标
[G_ex, G_in] = base.allocate([G_ex, G_inh], 10, 10, 20)

In [92]:
# -------initialization of network topology and synapses parameters----------
S_inE.connect(condition='j<0.3*N_post', p=p_inE)
S_inI.connect(condition='j<0.3*N_post', p=p_inI)
# 将neuron分配至3维空间，计算欧氏距离决定连接概率
S_EE.connect(condition='i != j', p='0.3*exp(-((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2)/R**2)')
S_EI.connect(p='0.2*exp(-((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2)/R**2)')
S_IE.connect(p='0.4*exp(-((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2)/R**2)')
S_II.connect(condition='i != j', p='0.1*exp(-((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2)/R**2)')
S_E_readout.connect(j='i')
S_I_readout.connect(j='i+n_ex')

S_inE.w = function.gamma(A_inE, S_inE.w.shape) # gamma分布随机变量采样
S_inI.w = function.gamma(A_inI, S_inI.w.shape)
S_EE.w = function.gamma(A_EE, S_EE.w.shape)
S_IE.w = function.gamma(A_IE, S_IE.w.shape)
S_EI.w = function.gamma(A_EI, S_EI.w.shape)
S_II.w = function.gamma(A_II, S_II.w.shape)

S_EE.pre.delay = '1.5*ms'
S_EI.pre.delay = '0.8*ms'
S_IE.pre.delay = '0.8*ms'
S_II.pre.delay = '0.8*ms'

In [93]:
# ------create network-------------
net = Network(collect())
net.store('init')

# ------run network-------------
stimulus = TimedArray(a[0][0], dt=Dt)
net.run(duration * Dt)
states = net.get_states()['neurongroup_read']['v']
net.restore('init')

# return (states, inputs[1])

In [96]:
states

array([0.49336867, 0.48960581, 0.41650842, ..., 0.        , 0.        ,
       0.        ])

In [51]:
X, Y, Z = 10, 10, 20
V = np.zeros((X, Y, Z), [('x', float), ('y', float), ('z', float)])
V['x'], V['y'], V['z'] = np.meshgrid(np.linspace(0, Y - 1, Y), 
                                     np.linspace(0, X - 1, X),
                                     np.linspace(0, Z - 1, Z))
V = V.reshape(X * Y * Z)
np.random.shuffle(V)
n = 0

In [53]:
print(V['x'])

[1. 5. 2. ... 6. 8. 3.]


In [50]:
np.linspace(0, Y - 1, Y)

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])